#Trabalho Final - Pesquisa Operacional
### Arthur Henrique da Silva - 20200077587
### Ludmila Vinólia Guimarães Gomes - 20210025065

---



#Conceito

Resumo do Funcionamento - Branch-and-Bound: o algoritmo Branch-and-Bound foi pensado para vencer a dificuldade existente nas resoluções de problemas de programação linear inteira (onde as variáveis a serem encontradas são obrigatoriamente valores inteiros) ao utilizar uma 'relaxação linear', em que o modelo de PI proposto se torna um modelo de programação linear apenas, ou seja, a restrição de integralidade das variáveis é retirada para que seja encontrada a solução. Com a solução, o polígono de soluções viáveis para o modelo PL encontrado é dividido em duas partes, devido à inclusão de restrições que limitam o espaço (Branch) e retiram uma região em que uma das variáveis assume valores não inteiros. Esse processo é realizado de forma cíclica até que um valor ótimo com variáveis de valores inteiros seja encontrado. O algoritmo para de executar quando os modelos relaxados não podem ser solucionados, devido às podas que podem ocorrer (Bounds):
 - por Limitante: o valor objetivo atual encontrado é menor que outros valores ótimos e possíveis (e não faz sentido buscar abaixo desse nó, pois os valores objetivos serão menores que o atual).
 - por Integralidade: os valores das variáveis são números inteiros (e o valor objetivo pode ser ótimo ou não).
 - por Inviabilidade: o polígono de soluções viáveis está 'vazio' e não pode ser encontrada solução para o modelo.

O presente projeto tem o objetivo de utilizar o algoritmo Branch-and-Bound, mas apenas na resolução de problemas de programação linear inteira binária. Assim, o modelo relaxado apresenta variáveis com valores reais no intervalo de 0 a 1 e a solução ótima procurada deve apresentar um valor binário (inteiro, claro), 0 ou 1.

Assim, é realizada a implementação do algoritmo de Branch-and-Bound para problemas de programação linear inteira binária usando Python e o pacote Mip, este usado para encontrar as soluções dos modelos relaxados.

Modelos de entrada aceitos: função objetivo de maximização, restrições de “menor ou igual” (com exceção das que definem o domı́nio das variáveis).

Saída gerada: para um caso de teste, o programa exibe em uma célula os resultados dos modelos dos nós, com valores para as variáveis e seus valores objetivos. Na célula seguinte, é exibida a solução ótima encontrada para esse caso de teste.

---

#Implementação

## Importação de pacotes

In [ ]:
#Instalação e importação do pacote mip
!pip install mip
from mip import *

## Funções Implementadas - Leitura do arquivo de entrada, resolução do modelo

In [ ]:
#Função simples que irá ser usada para extrair as informações dos arquivos de teste
def LeituraArquivo(nome):
  with open(nome, 'r') as f:
    #Lógica utilizada para separar as informações da primeira linha em suas respectivas variáveis
    linha1 = f.readline().split()
    vars = int(linha1[0]) #Quantidade de variáveis no modelo
    restr = int(linha1[1]) #Quantidade de restrições no modelo

    matriz = []

    #Construção de uma matriz com os coeficientes de cada restrição
    for linhas in f:
      num = list(map(int, linhas.split()))
      matriz.append(num)

  #Exibição das informações lidas no arquivo
  print(f"Qtd de variaveis: {vars}")
  print(f"Qtd de restricoes: {restr}")
  print("Coeficientes das restricoes:")
  for linhas in matriz:
    print(linhas)

  #Retorna os dados extraídos do arquivo para construção do modelo do caso de teste
  return vars, restr, matriz

In [ ]:
#Resolve o modelo e mostra os valores das variáveis encontradas na solução
def solve(model):
  status = model.optimize()

  #Retorna o status da solução do modelo, auxiliando a discernir se o modelo foi capaz de ser solucionado com valor ótimo ou se o mesmo não possui solução (infeasible)
  print(f"Status = {status}")
  if status != OptimizationStatus.OPTIMAL:
    return

  #Exibe o valor da função objetivo a partir dos valores encontrados para as variáveis
  print(f"Solution value  = {model.objective_value:.2f}\n")

  #Exibe o valor atribuído a cada variável do modelo
  print("Solution:")
  for v in model.vars:
    print(f"{v.name} = {v.x:.2f}")

  #Retorna uma lista de variáveis, em que cada variável (objeto) possui os atributos name(nome) e x(valor)
  return model.vars

In [ ]:
#Salva modelo em arquivo, e mostra o conteúdo
def save(model, filename):
  model.write(filename)
  with open(filename, "r") as f:
    print(f.read())

##Funções Implementadas - Funções com a lógica do algoritmo Branch-and-Bound

In [ ]:
#Verifica modelo e define como criar os nós filhos de um modelo resolvido (uso da regra da ramificação indicada na especificação do projeto)
def AtualizaModel(sol):
  #Verifica se o modelo que foi avaliado apresentou solução viável (ou seja, que não foi infeasible)
  #Se for infeasible, informa ao usuário que nenhuma solução ótima foi encontrada
  if sol is not None:
    #Remove os 0's dos valores de solução a serem comparados com o 0.5, tendo em vista que não são fracionários
    vars_sem_zero = [v for v in sol if v.x != 0]

    #Remove os 1's dos valores de solução a serem comparados com o 0.5, tendo em vista que não são fracionários
    vars_filtradas = [v for v in vars_sem_zero if v.x != 1]

    #Caso tenha variáveis a serem avaliadas, o algoritmo executa e retorna o índice da variável mais próxima de 0.5
    #Caso contrário, retorna -1 para indicar que não tem mais variáveis candidatas ou que não foi encontrado valor próximo
    if vars_filtradas:
      var_mais_prox = min(vars_filtradas, key = lambda v: abs(v.x - 0.5))

      if var_mais_prox:
        indice = int(var_mais_prox.name.split('_')[1])
        print(f'\n{var_mais_prox.name} = {var_mais_prox.x:.2f} é o mais próximo de 0.5\n')
        return indice
      else:
        return -1

    else:
      return -1
  else:
    print("Nenhuma solução ótima foi encontrada.\n")
    return -1

In [ ]:
#Estrutura básica da árvore, tendo um model associado a cada nó
class Arvore:
  def __init__(self, model):
    #Cada nó da árvore terá 4 informações relacionadas: o modelo a ser resolvido e seus resultados,
    #valor booleano para indicar se nó atual é nó terminal ou não e ponteiros para os nós esquerdo e direito
    self.model = model
    self.no_terminal = False
    self.esq = None
    self.dir = None

In [ ]:
#Utilizaremos uma fila para implementar a busca em largura dos nós da árvore
fila_nos_abertos = []

#Definição da estrutura de árvore para funcionamento do algoritmo Branch-and-Bound
class Arvore_BaB:
  def __init__(self, modelo_original):
    self.raiz = Arvore(modelo_original)

  #Método que realiza Branch, isto é, a ramificação de um nó
  def branch(self, no, indice):
    #Verifica se nó atual é nó terminal
    if no.no_terminal == True:
      #Estamos em um nó terminal
      print("Atingimos um nó terminal e não ocorre Branch!")
      return

    #Se o nó atual não for terminal, continua com fluxo de execução

    #Lógica para copiar o modelo do nó atual
    model_esq = no.model.copy()
    model_dir = no.model.copy()

    #Adição das restrições em cada nó dos diferentes ramos, seguindo as instruções da documentação
    model_esq += x[indice] == 0
    model_dir += x[indice] == 1

    #Construção da árvore, estabelecendo seu nó esquerdo e direito
    no.esq = Arvore(model_esq)
    no.dir = Arvore(model_dir)
    fila_nos_abertos.append(no.esq)
    fila_nos_abertos.append(no.dir)

In [ ]:
#Lista para armazenar todos os valores objetivos
l_objet = []

#Função para verificar se um nó sofreu algum dos três tipos de Bound (ou poda) e, em caso negativo, realiza Branch para o nó de entrada da função
def branch_and_bound(no):
  #Chamas as funções anteriormente definidas para capturar a solução encontrada e verificar qual índice deve ser atualizado
  sol = solve(no.model)
  indice = AtualizaModel(sol)

  #Verificação se houve Bound (poda por inviabilidade)
  if sol is None:
    print("INVIABILIDADE\n")
    no.no_terminal = True
    return True

  #Verificação se houve Bound (poda por integralidade)
  #Se não houver um índice válido, isto é, se todas as variáveis forem inteiras, temos a poda por integralidade!
  if indice == -1:
    print("INTEGRALIDADE\n")
    #Adiciona o valor objetivo a uma lista com todos os valores obtidos até agora
    l_objet.append(no.model.objective_value)
    no.no_terminal = True
    return True

  #Verificação se houve Bound (poda por limitante)
  #Aqui, vamos verificar se o valor objetivo atual é menor do que os que já existem na lista l_objet
  #Se for menor, então ocorre a poda por limitante, que significa que os valores objetivos encontrados do nó atual em diante não serão ótimos
  flag_limite = False
  for v_obj in l_objet:
    if v_obj > no.model.objective_value:
      flag_limite = True
      break

  #Quer dizer que o valor objetivo atual não é o ótimo e realizamos a pode por limitante

  #Verifica se flag_limite tem valor verdadeiro, o que significa que o valor objetivo atual é menor que os outros valores encontrados
  if flag_limite == True:
    print("LIMITANTE")
    no.no_terminal = True
    return True

  #Caso não tenha ocorrido Bound no nó, então processo de Branch é realizado
  arv.branch(no, indice)

##Casos de teste
---

**TESTE 1**

In [ ]:
from google.colab import files

#Faz o upload do arquivo
uploaded = files.upload()

#Pega o nome do arquivo que foi carregado
nome_arquivo = next(iter(uploaded))

Saving teste1.txt to teste1.txt


In [ ]:
#Variáveis para receber as informações extraídas do arquivo
vars, restr, matriz = LeituraArquivo(nome_arquivo)

Qtd de variaveis: 7
Qtd de restricoes: 11
Coeficientes das restricoes:
[2, 10, 8, 7, 10, 10, 6]
[5, 7, 8, 1, 7, 5, 6, 20]
[1, 6, 4, 9, 10, 6, 10, 30]
[4, 4, 4, 1, 5, 5, 10, 40]
[3, 10, 8, 1, 3, 3, 8, 30]
[10, 8, 9, 9, 7, 6, 10, 20]
[6, 6, 3, 6, 3, 7, 2, 80]
[7, 10, 7, 8, 7, 8, 7, 100]
[9, 8, 1, 1, 8, 10, 2, 90]
[1, 5, 3, 10, 2, 4, 9, 70]
[9, 6, 1, 4, 7, 5, 10, 60]
[5, 7, 4, 4, 3, 4, 10, 40]


In [ ]:
#Determinando que a função objetivo é de maximização e que o solver utilizado é CBC
model = Model(sense=MAXIMIZE, solver_name=CBC)

#Utilizamos CONTINUOUS para representar o modelo que passou por relaxação linear
#A quantidade de índices de x (variáveis) é dada justamente pelo valor de vars extraído do arquivo
x = {i: model.add_var(var_type=CONTINUOUS, name=f'x_{i}', lb=0.0) for i in range(vars) }

#Construção da função objetivo, sendo a mesma descrita por um somatório do coeficiente multiplicado por sua respectiva variável
soma = 0

#Percorre apenas os coeficientes da primeira linha da matriz
for i in range(len(matriz[0])):
  #Escrita do somatório da restrição, sendo matriz[0][i] o coeficiente e x[i] a variável
  soma += matriz[0][i] * x[i]
model.objective += soma #Atribui o somatório à função objetivo do modelo

#Construção das restrições do modelo
#Cada restrição é descrita também por um somatório do coeficiente multiplicado por sua respectiva variável <= a um valor fornecido no arquivo
#Começa a percorrer a matriz a partir da linha 1 (segunda linha), já que a primeira é dedicada a função objetivo
#Vale notar que i percorre as linhas, enquanto j percorre as colunas. Logo, as restrição são geradas para toda linha da matriz (a partir da 1)
for i in range(1, len(matriz)):
  soma = 0
  for j in range(len(matriz[i])):
    #Quando j for a última coluna da linha, isso significa que alcançou o valor que dita o <= da restrição
    #Por essa razão, soma <= matriz[i][j] representa o somatório <= valor da última coluna na linha
    if j == len(matriz[i]) - 1:
      model += soma <= matriz[i][j] #Atribui a restrição ao modelo
      break

    #Escrita do somatório da restrição, sendo matriz[i][j] o coeficiente e x[j] a variável
    soma += matriz[i][j] * x[j]

model.write("model.lp") # salva modelo em file
with open("model.lp") as f: # lê e exibe conteúdo do file
  print(f.read())

\Problem name: 

Minimize
OBJROW: -2 x_0 -10 x_1 -8 x_2 -7 x_3 -10 x_4 -10 x_5 -6 x_6
Subject To
constr(0):  5 x_0 + 7 x_1 + 8 x_2 + x_3 + 7 x_4 + 5 x_5 + 6 x_6 <= 20
constr(1):  x_0 + 6 x_1 + 4 x_2 + 9 x_3 + 10 x_4 + 6 x_5 + 10 x_6 <= 30
constr(2):  4 x_0 + 4 x_1 + 4 x_2 + x_3 + 5 x_4 + 5 x_5 + 10 x_6 <= 40
constr(3):  3 x_0 + 10 x_1 + 8 x_2 + x_3 + 3 x_4 + 3 x_5 + 8 x_6 <= 30
constr(4):  10 x_0 + 8 x_1 + 9 x_2 + 9 x_3 + 7 x_4 + 6 x_5 + 10 x_6 <= 20
constr(5):  6 x_0 + 6 x_1 + 3 x_2 + 6 x_3 + 3 x_4 + 7 x_5 + 2 x_6 <= 80
constr(6):  7 x_0 + 10 x_1 + 7 x_2 + 8 x_3 + 7 x_4 + 8 x_5 + 7 x_6 <= 100
constr(7):  9 x_0 + 8 x_1 + x_2 + x_3 + 8 x_4 + 10 x_5 + 2 x_6 <= 90
constr(8):  x_0 + 5 x_1 + 3 x_2 + 10 x_3 + 2 x_4 + 4 x_5 + 9 x_6 <= 70
constr(9):  9 x_0 + 6 x_1 + x_2 + 4 x_3 + 7 x_4 + 5 x_5 + 10 x_6 <= 60
constr(10):  5 x_0 + 7 x_1 + 4 x_2 + 4 x_3 + 3 x_4 + 4 x_5 + 10 x_6 <= 40
Bounds
End



In [ ]:
arv = Arvore_BaB(model)
branch_and_bound(arv.raiz)

while fila_nos_abertos:
  novo_no = fila_nos_abertos.pop(0)
  branch_and_bound(novo_no)

Status = OptimizationStatus.OPTIMAL
Solution value  = 33.33

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 0.00
x_3 = 0.00
x_4 = 0.00
x_5 = 3.33
x_6 = 0.00

x_5 = 3.33 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 28.57

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 0.00
x_3 = 0.00
x_4 = 2.86
x_5 = 0.00
x_6 = 0.00

x_4 = 2.86 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 30.00

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 0.00
x_3 = 0.00
x_4 = 2.00
x_5 = 1.00
x_6 = 0.00

x_4 = 2.00 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 25.00

Solution:
x_0 = 0.00
x_1 = 2.50
x_2 = 0.00
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00

x_1 = 2.50 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 26.25

Solution:
x_0 = 0.00
x_1 = 1.62
x_2 = 0.00
x_3 = 0.00
x_4 = 1.00
x_5 = 0.00
x_6 = 0.00

x_1 = 1.62 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 27.50

Solution:
x_0 

In [ ]:
maior = 0
for i in l_objet:
  if i > maior:
    maior = i
print("VALOR ÓTIMO ENCONTRADO:", maior)

VALOR ÓTIMO ENCONTRADO: 20.0


**TESTE 2**

In [ ]:
from google.colab import files

#Faz o upload do arquivo
uploaded = files.upload()

#Pega o nome do arquivo que foi carregado
nome_arquivo = next(iter(uploaded))

Saving teste2.txt to teste2.txt


In [ ]:
vars, restr, matriz = LeituraArquivo(nome_arquivo)

Qtd de variaveis: 9
Qtd de restricoes: 9
Coeficientes das restricoes:
[7, 7, 7, 5, 8, 8, 9, 10, 7]
[1, 3, 1, 3, 3, 7, 2, 1, 4, 80]
[7, 6, 10, 1, 7, 2, 2, 7, 1, 90]
[3, 2, 1, 3, 3, 2, 1, 6, 5, 10]
[10, 8, 3, 6, 10, 7, 3, 10, 4, 30]
[2, 8, 6, 5, 6, 6, 9, 7, 2, 80]
[3, 10, 1, 9, 2, 7, 7, 9, 10, 90]
[1, 7, 10, 10, 5, 2, 9, 10, 2, 20]
[10, 3, 2, 3, 10, 2, 1, 9, 7, 10]
[2, 1, 7, 10, 1, 1, 2, 1, 1, 50]


In [ ]:
model2 = Model(sense=MAXIMIZE, solver_name=CBC)

x = {i: model2.add_var(var_type=CONTINUOUS, name=f'x_{i}', lb=0.0) for i in range(vars) }

soma = 0
for i in range(len(matriz[0])):
  soma += matriz[0][i] * x[i]
model2.objective += soma

for i in range(1, len(matriz)):
  soma = 0
  for j in range(len(matriz[i])):
    if j == len(matriz[i]) - 1:
      model2 += soma <= matriz[i][j]
      break

    soma += matriz[i][j] * x[j]


model2.write("model2.lp")
with open("model2.lp") as f:
  print(f.read())

\Problem name: 

Minimize
OBJROW: -7 x_0 -7 x_1 -7 x_2 -5 x_3 -8 x_4 -8 x_5 -9 x_6 -10 x_7 -7 x_8
Subject To
constr(0):  x_0 + 3 x_1 + x_2 + 3 x_3 + 3 x_4 + 7 x_5 + 2 x_6 + x_7 + 4 x_8 <= 80
constr(1):  7 x_0 + 6 x_1 + 10 x_2 + x_3 + 7 x_4 + 2 x_5 + 2 x_6 + 7 x_7 + x_8 <= 90
constr(2):  3 x_0 + 2 x_1 + x_2 + 3 x_3 + 3 x_4 + 2 x_5 + x_6 + 6 x_7 + 5 x_8 <= 10
constr(3):  10 x_0 + 8 x_1 + 3 x_2 + 6 x_3 + 10 x_4 + 7 x_5 + 3 x_6 + 10 x_7 + 4 x_8 <= 30
constr(4):  2 x_0 + 8 x_1 + 6 x_2 + 5 x_3 + 6 x_4 + 6 x_5 + 9 x_6 + 7 x_7 + 2 x_8 <= 80
constr(5):  3 x_0 + 10 x_1 + x_2 + 9 x_3 + 2 x_4 + 7 x_5 + 7 x_6 + 9 x_7 + 10 x_8 <= 90
constr(6):  x_0 + 7 x_1 + 10 x_2 + 10 x_3 + 5 x_4 + 2 x_5 + 9 x_6 + 10 x_7 + 2 x_8 <= 20
constr(7):  10 x_0 + 3 x_1 + 2 x_2 + 3 x_3 + 10 x_4 + 2 x_5 + x_6 + 9 x_7 + 7 x_8 <= 10
constr(8):  2 x_0 + x_1 + 7 x_2 + 10 x_3 + x_4 + x_5 + 2 x_6 + x_7 + x_8 <= 50
Bounds
End



In [ ]:
l_objet = []
fila_nos_abertos = []

arv = Arvore_BaB(model2)
branch_and_bound(arv.raiz)

while fila_nos_abertos:
  novo_no = fila_nos_abertos.pop(0)
  branch_and_bound(novo_no)

Status = OptimizationStatus.OPTIMAL
Solution value  = 42.49

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 0.00
x_3 = 0.00
x_4 = 0.00
x_5 = 3.57
x_6 = 1.38
x_7 = 0.00
x_8 = 0.21

x_8 = 0.21 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 42.11

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 0.00
x_3 = 0.00
x_4 = 0.00
x_5 = 3.68
x_6 = 1.40
x_7 = 0.00
x_8 = 0.00

x_6 = 1.40 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 28.38

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 0.00
x_3 = 0.00
x_4 = 0.00
x_5 = 0.56
x_6 = 1.88
x_7 = 0.00
x_8 = 1.00

x_5 = 0.56 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 38.75

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 1.25
x_3 = 0.00
x_4 = 0.00
x_5 = 3.75
x_6 = 0.00
x_7 = 0.00
x_8 = 0.00

x_2 = 1.25 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 41.14

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 0.36
x_3 = 0.00
x_4 = 0.00
x_5 = 3.70
x_6 = 1.00
x_7 = 0.00
x_8 = 0.00

x

In [ ]:
maior = 0
for i in l_objet:
  if i > maior:
    maior = i
print("VALOR ÓTIMO ENCONTRADO:", maior)

VALOR ÓTIMO ENCONTRADO: 24.0


**TESTE 3**

In [ ]:
from google.colab import files

#Faz o upload do arquivo
uploaded = files.upload()

#Pega o nome do arquivo que foi carregado
nome_arquivo = next(iter(uploaded))

Saving teste3.txt to teste3.txt


In [ ]:
vars, restr, matriz = LeituraArquivo(nome_arquivo)

Qtd de variaveis: 9
Qtd de restricoes: 12
Coeficientes das restricoes:
[7, 9, 10, 3, 6, 1, 9, 8, 8]
[2, 1, 9, 6, 3, 6, 10, 9, 1, 60]
[8, 6, 6, 5, 2, 2, 4, 3, 6, 80]
[8, 1, 3, 7, 1, 4, 8, 3, 4, 30]
[6, 3, 9, 5, 9, 6, 9, 9, 6, 40]
[10, 8, 8, 7, 10, 10, 9, 9, 3, 20]
[10, 10, 10, 9, 10, 1, 8, 3, 10, 90]
[10, 5, 8, 2, 7, 8, 6, 2, 2, 90]
[7, 9, 1, 9, 5, 8, 5, 9, 5, 80]
[2, 4, 6, 1, 7, 9, 10, 1, 7, 80]
[8, 10, 2, 6, 7, 2, 2, 4, 9, 10]
[10, 9, 1, 4, 2, 4, 4, 8, 2, 30]
[9, 4, 1, 8, 9, 5, 10, 5, 8, 30]


In [ ]:
model3 = Model(sense=MAXIMIZE, solver_name=CBC)

x = {i: model3.add_var(var_type=CONTINUOUS, name=f'x_{i}', lb=0.0) for i in range(vars) }

soma = 0
for i in range(len(matriz[0])):
  soma += matriz[0][i] * x[i]
model3.objective += soma

for i in range(1, len(matriz)):
  soma = 0
  for j in range(len(matriz[i])):
    if j == len(matriz[i]) - 1:
      model3 += soma <= matriz[i][j]
      break

    soma += matriz[i][j] * x[j]


model3.write("model3.lp")
with open("model3.lp") as f:
  print(f.read())

\Problem name: 

Minimize
OBJROW: -7 x_0 -9 x_1 -10 x_2 -3 x_3 -6 x_4 - x_5 -9 x_6 -8 x_7 -8 x_8
Subject To
constr(0):  2 x_0 + x_1 + 9 x_2 + 6 x_3 + 3 x_4 + 6 x_5 + 10 x_6 + 9 x_7 + x_8 <= 60
constr(1):  8 x_0 + 6 x_1 + 6 x_2 + 5 x_3 + 2 x_4 + 2 x_5 + 4 x_6 + 3 x_7 + 6 x_8 <= 80
constr(2):  8 x_0 + x_1 + 3 x_2 + 7 x_3 + x_4 + 4 x_5 + 8 x_6 + 3 x_7 + 4 x_8 <= 30
constr(3):  6 x_0 + 3 x_1 + 9 x_2 + 5 x_3 + 9 x_4 + 6 x_5 + 9 x_6 + 9 x_7 + 6 x_8 <= 40
constr(4):  10 x_0 + 8 x_1 + 8 x_2 + 7 x_3 + 10 x_4 + 10 x_5 + 9 x_6 + 9 x_7 + 3 x_8 <= 20
constr(5):  10 x_0 + 10 x_1 + 10 x_2 + 9 x_3 + 10 x_4 + x_5 + 8 x_6 + 3 x_7 + 10 x_8 <= 90
constr(6):  10 x_0 + 5 x_1 + 8 x_2 + 2 x_3 + 7 x_4 + 8 x_5 + 6 x_6 + 2 x_7 + 2 x_8 <= 90
constr(7):  7 x_0 + 9 x_1 + x_2 + 9 x_3 + 5 x_4 + 8 x_5 + 5 x_6 + 9 x_7 + 5 x_8 <= 80
constr(8):  2 x_0 + 4 x_1 + 6 x_2 + x_3 + 7 x_4 + 9 x_5 + 10 x_6 + x_7 + 7 x_8 <= 80
constr(9):  8 x_0 + 10 x_1 + 2 x_2 + 6 x_3 + 7 x_4 + 2 x_5 + 2 x_6 + 4 x_7 + 9 x_8 <= 10
constr(10):  10 

In [ ]:
l_objet = []
fila_nos_abertos = []

arv = Arvore_BaB(model3)
branch_and_bound(arv.raiz)

while fila_nos_abertos:
  novo_no = fila_nos_abertos.pop(0)
  branch_and_bound(novo_no)

Status = OptimizationStatus.OPTIMAL
Solution value  = 27.58

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 2.27
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.00
x_8 = 0.61

x_8 = 0.61 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 25.00

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 2.50
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.00
x_8 = 0.00

x_2 = 2.50 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 13.00

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 0.50
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.00
x_8 = 1.00

x_2 = 0.50 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 20.68

Solution:
x_0 = 0.00
x_1 = 0.68
x_2 = 0.00
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 1.62
x_7 = 0.00
x_8 = 0.00

x_1 = 0.68 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 22.65

Solution:
x_0 = 0.00
x_1 = 0.65
x_2 = 1.00
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.76
x_7 = 0.00
x_8 = 0.00

x

In [ ]:
maior = 0
for i in l_objet:
  if i > maior:
    maior = i
print("VALOR ÓTIMO ENCONTRADO:", maior)

VALOR ÓTIMO ENCONTRADO: 19.0


**TESTE 4**

In [ ]:
from google.colab import files

#Faz o upload do arquivo
uploaded = files.upload()

#Pega o nome do arquivo que foi carregado
nome_arquivo = next(iter(uploaded))

Saving teste4.txt to teste4.txt


In [ ]:
vars, restr, matriz = LeituraArquivo(nome_arquivo)

Qtd de variaveis: 9
Qtd de restricoes: 9
Coeficientes das restricoes:
[9, 7, 10, 7, 9, 6, 8, 4, 9]
[4, 9, 4, 1, 9, 6, 3, 6, 1, 40]
[3, 7, 8, 7, 6, 3, 5, 9, 4, 80]
[9, 3, 6, 5, 7, 1, 1, 3, 9, 40]
[5, 9, 6, 5, 9, 7, 8, 7, 8, 10]
[7, 7, 4, 1, 3, 4, 8, 1, 9, 10]
[1, 6, 6, 1, 6, 7, 3, 8, 7, 10]
[6, 6, 8, 6, 10, 8, 1, 4, 4, 70]
[9, 1, 9, 7, 10, 5, 6, 2, 5, 10]
[2, 7, 6, 5, 1, 1, 9, 2, 1, 20]


In [ ]:
model4 = Model(sense=MAXIMIZE, solver_name=CBC)

x = {i: model4.add_var(var_type=CONTINUOUS, name=f'x_{i}', lb=0.0) for i in range(vars) }

soma = 0
for i in range(len(matriz[0])):
  soma += matriz[0][i] * x[i]
model4.objective += soma

for i in range(1, len(matriz)):
  soma = 0
  for j in range(len(matriz[i])):
    if j == len(matriz[i]) - 1:
      model4 += soma <= matriz[i][j]
      break

    soma += matriz[i][j] * x[j]


model4.write("model4.lp")
with open("model4.lp") as f:
  print(f.read())

\Problem name: 

Minimize
OBJROW: -9 x_0 -7 x_1 -10 x_2 -7 x_3 -9 x_4 -6 x_5 -8 x_6 -4 x_7 -9 x_8
Subject To
constr(0):  4 x_0 + 9 x_1 + 4 x_2 + x_3 + 9 x_4 + 6 x_5 + 3 x_6 + 6 x_7 + x_8 <= 40
constr(1):  3 x_0 + 7 x_1 + 8 x_2 + 7 x_3 + 6 x_4 + 3 x_5 + 5 x_6 + 9 x_7 + 4 x_8 <= 80
constr(2):  9 x_0 + 3 x_1 + 6 x_2 + 5 x_3 + 7 x_4 + x_5 + x_6 + 3 x_7 + 9 x_8 <= 40
constr(3):  5 x_0 + 9 x_1 + 6 x_2 + 5 x_3 + 9 x_4 + 7 x_5 + 8 x_6 + 7 x_7 + 8 x_8 <= 10
constr(4):  7 x_0 + 7 x_1 + 4 x_2 + x_3 + 3 x_4 + 4 x_5 + 8 x_6 + x_7 + 9 x_8 <= 10
constr(5):  x_0 + 6 x_1 + 6 x_2 + x_3 + 6 x_4 + 7 x_5 + 3 x_6 + 8 x_7 + 7 x_8 <= 10
constr(6):  6 x_0 + 6 x_1 + 8 x_2 + 6 x_3 + 10 x_4 + 8 x_5 + x_6 + 4 x_7 + 4 x_8 <= 70
constr(7):  9 x_0 + x_1 + 9 x_2 + 7 x_3 + 10 x_4 + 5 x_5 + 6 x_6 + 2 x_7 + 5 x_8 <= 10
constr(8):  2 x_0 + 7 x_1 + 6 x_2 + 5 x_3 + x_4 + x_5 + 9 x_6 + 2 x_7 + x_8 <= 20
Bounds
End



In [ ]:
l_objet = []
fila_nos_abertos = []

arv = Arvore_BaB(model4)
branch_and_bound(arv.raiz)

while fila_nos_abertos:
  novo_no = fila_nos_abertos.pop(0)
  branch_and_bound(novo_no)

Status = OptimizationStatus.OPTIMAL
Solution value  = 13.57

Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 0.71
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.00
x_8 = 0.71

x_2 = 0.71 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 13.08

Solution:
x_0 = 0.38
x_1 = 0.00
x_2 = 0.00
x_3 = 0.38
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.00
x_8 = 0.77

x_3 = 0.38 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 13.49

Solution:
x_0 = 0.00
x_1 = 0.32
x_2 = 1.00
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.00
x_8 = 0.14

x_1 = 0.32 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 12.69

Solution:
x_0 = 1.02
x_1 = 0.38
x_2 = 0.00
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.22
x_8 = 0.00

x_1 = 0.38 é o mais próximo de 0.5

Status = OptimizationStatus.OPTIMAL
Solution value  = 12.54

Solution:
x_0 = 0.00
x_1 = 0.03
x_2 = 0.00
x_3 = 1.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.00
x_8 = 0.59

x

In [ ]:
maior = 0
for i in l_objet:
  if i > maior:
    maior = i
print("VALOR ÓTIMO ENCONTRADO:", maior)

VALOR ÓTIMO ENCONTRADO: 10.0
